In [8]:
!pip install -r ./requirements.txt

  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Ignored the following yanked versions: 1.0.14, 1.0.102, 1.0.120, 1.0.139, 1.0.162, 1.0.179, 1.0.180, 1.0.248, 1.0.250, 1.0.255, 1.0.260, 1.0.288, 1.0.313, 1.0.324, 1.0.335
ERROR: Could not find a version that satisfies the requirement ibm_watson_machine_learning==1.0.367 (from versions: 

In [9]:
import numpy as np
print(np.__version__)

2.0.2


In [6]:
import os
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as MetaNames

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
def chat_with_bot(model):
    print("Welcome to the WatsonX Chatbot! (Type 'quit' to exit)")
    print("-" * 50)
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() == 'quit':
            print("\nGoodbye!")
            break
            
    
        prompt = f"Human: {user_input}\n\nAssistant:"
        
    
        parameters = {
            MetaNames.DECODING_METHOD: "greedy",
            MetaNames.MAX_NEW_TOKENS: 100,
            MetaNames.MIN_NEW_TOKENS: 1,
            MetaNames.STOP_SEQUENCES: ["Human:", "\n\n"]
        }
        
        try:
            response = model.generate(prompt, parameters)
        
            generated_text = response['results'][0]['generated_text'].strip()
            print("\nBot:", generated_text)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [ ]:
load_dotenv()

model_id = "ibm/granite-34b-code-instruct" 
credentials = {
    "apikey": os.getenv("WATSONX_API_KEY"),
    "url": "https://us-south.ml.cloud.ibm.com"
}

model = Model(
    model_id=model_id,
    credentials=credentials,
    project_id=os.getenv("WATSONX_PROJECT_ID")
)
chat_with_bot(model) 